In [2]:
from __future__ import annotations

import ase
from ase.calculators.lj import LennardJones as ASE_LennardJones
from graph_pes.models import LennardJones as GraphPES_LennardJones
from graph_pes.utils.calculator import GraphPESCalculator

lj_properties = {
    "rc": 2.0,
    "sigma": 1.0,
    "epsilon": 1.0,
    "smooth": False,
}

ase_lj_calc = ASE_LennardJones(**lj_properties)
graph_pes_lj_calc = GraphPESCalculator(
    model=GraphPES_LennardJones.from_ase(**lj_properties)
)

dimer = ase.Atoms(
    positions=[
        [0, 0, 0],
        [0, 0, 0.98],
    ]
)

ase_lj_calc.get_potential_energy(dimer)

0.643428299130453

In [3]:
graph_pes_lj_calc.get_potential_energy(dimer)

0.643427848815918

In [4]:
ase_lj_calc.get_forces(dimer)

array([[  0.        ,   0.        , -34.77113701],
       [  0.        ,   0.        ,  34.77113701]])

In [6]:
graph_pes_lj_calc.get_forces(dimer)


array([[ -0.      ,  -0.      , -34.771133],
       [ -0.      ,  -0.      ,  34.771133]], dtype=float32)

In [20]:
import numpy as np
from ase.stress import voigt_6_to_full_3x3_stress


def get_stress_matrix(calculator, structure):
    stress = calculator.get_stress(structure)
    if stress.shape != (3, 3):
        stress = voigt_6_to_full_3x3_stress(stress)
    return np.round(stress, 3)


cell_size = 1
periodic_structure = ase.Atoms(
    positions=[
        [cell_size / 2, cell_size / 2, cell_size / 2],
    ],
    cell=[cell_size, cell_size, cell_size],
    pbc=True,
)

get_stress_matrix(ase_lj_calc, periodic_structure)

array([[-18.039,  -0.   ,   0.   ],
       [ -0.   , -18.039,   0.   ],
       [  0.   ,   0.   , -18.039]])

In [21]:
get_stress_matrix(graph_pes_lj_calc, periodic_structure)

array([[-18.039,   0.   ,   0.   ],
       [  0.   , -18.039,  -0.   ],
       [  0.   ,  -0.   , -18.039]])

## Finite-differences

The force on atom $i$ along direction $d$ is given by:

$$
F^{(d)}_i = -\frac{\partial E}{\partial R^{(d)}_i}
$$

which we can estimate by perturbing $R^{(d)}_i$ and calculating the
finite-difference:

$$
F^{(d)}_i \approx \frac{E\left(R^{(d)}_i + \epsilon\right) - E\left(R^{(d)}_i - \epsilon\right)}{2\epsilon}
$$

We can also estimate the stress tensor in a simlar fashion.


In [9]:
from typing import Callable, Literal


def finite_difference_force_estimate(
    energy_function: Callable[[ase.Atoms], float],
    atoms: ase.Atoms,
    atom_index: int,
    direction: Literal["x", "y", "z"],
    epsilon: float = 1e-4,
) -> float:
    direction_index = {"x": 0, "y": 1, "z": 2}[direction]

    copy1 = atoms.copy()
    copy1.positions[atom_index][direction_index] += epsilon
    copy2 = atoms.copy()
    copy2.positions[atom_index][direction_index] -= epsilon

    f1 = energy_function(copy1)
    f2 = energy_function(copy2)
    return (f2 - f1) / (2 * epsilon)


def finite_difference_stress_component(
    energy_function: Callable[[ase.Atoms], float],
    atoms: ase.Atoms,
    component: tuple[int, int],
    epsilon: float = 1e-6,
) -> float:
    i, j = component

    scaling = np.eye(3)
    if i == j:
        scaling[i, j] += epsilon
    else:
        scaling[i, j] = scaling[j, i] = epsilon / 2

    copy = atoms.copy()
    copy.positions = atoms.positions @ scaling
    copy.cell = atoms.cell @ scaling

    f1 = energy_function(copy)
    f2 = energy_function(atoms)
    return (f1 - f2) / epsilon / atoms.get_volume()


def finite_difference_stress_estimate(
    energy_function: Callable[[ase.Atoms], float],
    atoms: ase.Atoms,
    epsilon: float = 1e-6,
) -> np.ndarray:
    stress = np.zeros((3, 3))
    for i in range(3):
        for j in range(3):
            stress[i, j] = finite_difference_stress_component(
                energy_function, atoms, (i, j), epsilon
            )
    return stress

Lets check that our autograd-based implementation matches the finite-difference estimates:

In [22]:
from graph_pes.models import SchNet

model = SchNet(cutoff=3.0)
schnet_calc = GraphPESCalculator(model)

cell_size = 4
n_atoms = 10
random_structure = ase.Atoms(
    positions=np.random.RandomState(42).rand(n_atoms, 3) * cell_size,
    cell=[cell_size, cell_size, cell_size],
    pbc=True,
)

In [23]:
atom_index = 0
direction = "x"

fd_estimate = finite_difference_force_estimate(
    schnet_calc.get_potential_energy,
    random_structure,
    atom_index,
    direction,
    epsilon=1e-3,
)
autograd_value = schnet_calc.get_forces(random_structure)[atom_index][
    {"x": 0, "y": 1, "z": 2}[direction]
]

np.round(fd_estimate, 6), np.round(autograd_value, 6)

(-0.034377, -0.034358)

In [24]:
# finite-difference stress
fd_stress = finite_difference_stress_estimate(
    schnet_calc.get_potential_energy, random_structure, epsilon=1e-3
)
np.round(fd_stress, 6)


array([[ 0.001911,  0.002092, -0.003447],
       [ 0.002092,  0.003045, -0.00231 ],
       [-0.003447, -0.00231 ,  0.005248]])

In [25]:
# autograd stress
autograd_stress = voigt_6_to_full_3x3_stress(
    schnet_calc.get_stress(random_structure)
)
np.round(autograd_stress, 6)

array([[ 0.001928,  0.002072, -0.003426],
       [ 0.002072,  0.003042, -0.002297],
       [-0.003426, -0.002297,  0.005268]])